In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

from nltk.corpus import wordnet

import os

In [2]:
DATA_DIR = os.path.join('..', 'data')

TEST_IMAGE_DIR = os.path.join(DATA_DIR, 'test-images')



In [3]:
dog_images = ['walking-the-dog.jpg', 'yellow-dog.jpeg']

not_dog_images = ['front-yard.jpg', 'living-room.jpg']


In [4]:
model = ResNet50(weights='imagenet')



In [5]:
def get_test_image_path(file_name):
    return os.path.join(TEST_IMAGE_DIR, file_name)

In [6]:
def name_exists_in_hypernym(name, syn):
    hypernyms = syn.hypernyms()
    
    if hypernyms is None or len(hypernyms) == 0:
        return False
    
    for hypernym in hypernyms:
        hypernym_name = syn.name()
        if name in hypernym_name:
            return True
        if name_exists_in_hypernym(name, hypernym):
            return True
    
    return False

In [7]:
def is_dog_in_image_prediction(prediction):
    prediction_name = prediction[1]
    
    if 'dog' in prediction_name:
        return True
    
    synsets = wordnet.synsets(prediction_name)
    if synsets is None or len(synsets) == 0:
        return False
    
    for syn in synsets:
        if name_exists_in_hypernym('dog', syn):
            return True
    return False

    
    

In [8]:
def display_image(image_name):
    image_filepath = get_test_image_path(image_name)

    return image.load_img(image_filepath, target_size=(224, 224))

In [9]:
def get_image_predictions(image_name):
    image_filepath = get_test_image_path(image_name)
        
    img = image.load_img(image_filepath, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    
    return  decode_predictions(preds, top=3)[0]

def predict_images():
    for image_name in dog_images:
        predictions = get_image_predictions(image_name)
        
        print('Predicted:', predictions)
        
predict_images()

Predicted: [('n02088238', 'basset', 0.4150713), ('n02088466', 'bloodhound', 0.0665933), ('n02088094', 'Afghan_hound', 0.03386655)]
Predicted: [('n02099601', 'golden_retriever', 0.4936183), ('n02099712', 'Labrador_retriever', 0.20975316), ('n02088466', 'bloodhound', 0.15128355)]


In [10]:
def is_dog_in_image(image_name):
    predictions = get_image_predictions(image_name)
    
    for prediction in predictions:
        if is_dog_in_image_prediction(prediction):
            return True
    return False
    
    
for dog_image_path in dog_images:
    print(is_dog_in_image(dog_image_path))

True
True


In [11]:
for not_dog_image in not_dog_images:
    print(get_image_predictions(not_dog_image))
    print(is_dog_in_image(not_dog_image))

[('n03891251', 'park_bench', 0.30316633), ('n03899768', 'patio', 0.16220686), ('n09332890', 'lakeside', 0.117380574)]
False
[('n04239074', 'sliding_door', 0.30991945), ('n03529860', 'home_theater', 0.19330558), ('n04590129', 'window_shade', 0.064584784)]
False


In [12]:
# wordnet.synsets('golden_retriever')[0].hypernyms()[0].hypernyms()[0].hypernyms()[0].hypernyms()[0].name()